In [35]:
import json
import pickle 
import numpy as np
import random
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to C:\Users\Yash
[nltk_data]     Phatak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Yash
[nltk_data]     Phatak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
print(np.__version__)

1.23.5


In [37]:
import tensorflow as tf

In [38]:
print(np.__version__)

1.23.5


In [39]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout,ReLU
from keras.optimizers import SGD

Creating lists for Processing 

In [40]:
words = []
classes = []
documents = []
ignore_words =['?','!']

In [41]:
data = open('intents.json').read()
intents = json.loads(data)

In [42]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenisation of each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #adding documents
        documents.append((w,intent['tag']))
        
        #adding classes to the class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [43]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower(),pos='v') for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [44]:
words

["'m",
 "'s",
 '(',
 ')',
 ',',
 '.',
 '2',
 'a',
 'about',
 'accept',
 'access',
 'achieve',
 'adopt',
 'adoption',
 'advantage',
 'affect',
 'airdrop',
 'airdrops',
 'allow',
 'altcoins',
 'alternative',
 'aml',
 'an',
 'analyze',
 'and',
 'anonymity',
 'any',
 'anyone',
 'applications',
 'approach',
 'art',
 'as',
 'asset',
 'assets',
 'at',
 'atomic',
 'autonomous',
 'avoid',
 'awesome',
 'bank',
 'be',
 'beginner',
 'behind',
 'benefit',
 'best',
 'between',
 'bitcoin',
 'bitcoins',
 'bite',
 'blockchain',
 'blockchain-based',
 'break',
 'btc',
 'businesses',
 'buy',
 'by',
 'bye',
 'can',
 'cap',
 'categories',
 'cause',
 'cbdc',
 'cbdcs',
 'central',
 'centralization',
 'centralize',
 'change',
 'charge',
 'chat',
 'choose',
 'coin',
 'common',
 'communities',
 'community',
 'compare',
 'compatibility',
 'compliance',
 'con',
 'concept',
 'concern',
 'conduct',
 'consensus',
 'consider',
 'considerations',
 'contract',
 'could',
 'countries',
 'country',
 'course',
 'cross-borde

In [45]:
classes = sorted(list(set(classes)))
classes

['Adoption',
 'Altcoins',
 'Best and Safest Crypto',
 'Bitcoin',
 'BitcoinDesc',
 'Blockchain',
 'Blockchain Use Cases',
 'BlockchainExplained',
 'Buying Cryptocurrencies',
 'CBDC',
 'Central Bank Digital Currencies (CBDCs)',
 'Centralization vs. Decentralization',
 'ChoosingExchange',
 'Cross-Border Payments',
 'Cross-Chain Swaps',
 'Crypto and Gaming',
 'CryptoBasics',
 'CryptoRisks',
 'Cryptocurrency Derivatives',
 'Cryptocurrency Education',
 'Cryptocurrency Regulation',
 'Cryptocurrency Security',
 'Cryptocurrency Taxes',
 'CryptocurrencyLaws',
 'CryptocurrencyValue',
 'Decentralized Autonomous Organizations (DAOs)',
 'Decentralized Exchanges (DEXs)',
 'Decentralized Finance (DeFi)',
 'Digital Identity',
 'Diversification',
 'EarningCrypto',
 'EducationalResources',
 'Environmental Impact',
 'Ethereum',
 'EthereumDesc',
 'Examples',
 'Exchanges',
 'ExplainSimply',
 'Forking',
 'Future Trends',
 'Getting Started with Investing',
 'Governance',
 'How Crypto Works',
 'How Mining Work

In [46]:
#Creating a Pickle file for classes and words
pickle.dump(words,open('words.pkl',mode='wb'))
pickle.dump(classes,open('classes.pkl',mode='wb'))

In [47]:
documents

[(['Hiii'], 'greeting'),
 (['Hi'], 'greeting'),
 (['Heylos'], 'greeting'),
 (['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Hola', '!'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'),
 (['Till', 'next', 'time'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Awesome', ',', 'thanks'], 'thanks'),
 (['Thanks', 'for', 'helping', 'me'], 'thanks'),
 (['How', 'you', 'could', 'help', 'me', '?'], 'options'),
 (['What', 'you', 'can', 'do', '?'], 'options'),
 (['What', 'help', 'you', 'provide', '?'], 'options'),
 (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'),
 (['What', 'support', 'is', 'offered'], 'options'),
 (['Crypto', '?'], 'In

In [48]:
#building the dl model
training =[]
output_empty = [0]*len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words if word not in ignore_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag,output_row])
#shuffle and conversion to np array
random.shuffle(training)
training = np.array(training)

C:\Users\Yash Phatak\AppData\Local\Temp\ipykernel_15116\427115024.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [49]:
training

array([[list([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [50]:
#train split for patterns and tags
train_x = list(training[:,0]) #for patterns
train_y = list(training[:,1]) #intents or tags
print("Training data gg")

Training data gg


In [51]:
# 3 layers - 128 neurons - 64 neurons - intents number
model =  Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
#Compile model.Stochastic gradient descent with Nesterov accelarated gradient gives good results for this model
sgd = SGD(learning_rate=0.01,weight_decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('krypto_model.h5', hist)
print("Model Created.GG!")

Epoch 1/200


79/79 [==============================] - 1s 3ms/step - loss: 4.7719 - accuracy: 0.0179
Epoch 2/200
79/79 [==============================] - 0s 3ms/step - loss: 4.7417 - accuracy: 0.0306
Epoch 3/200
79/79 [==============================] - 0s 2ms/step - loss: 4.7277 - accuracy: 0.0255
Epoch 4/200
79/79 [==============================] - 0s 3ms/step - loss: 4.6940 - accuracy: 0.0510
Epoch 5/200
79/79 [==============================] - 0s 2ms/step - loss: 4.6615 - accuracy: 0.0383
Epoch 6/200
79/79 [==============================] - 0s 2ms/step - loss: 4.6246 - accuracy: 0.0459
Epoch 7/200
79/79 [==============================] - 0s 2ms/step - loss: 4.5722 - accuracy: 0.0536
Epoch 8/200
79/79 [==============================] - 0s 3ms/step - loss: 4.5552 - accuracy: 0.0536
Epoch 9/200
79/79 [==============================] - 0s 2ms/step - loss: 4.4844 - accuracy: 0.0485
Epoch 10/200
79/79 [==============================] - 0s 2ms/step - loss: 4.3775 - accuracy: 0.0689
Epoch 11/200
79/79 [=